In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests
warnings.filterwarnings("ignore")

# Correlation Between Metabolites/Cytokines and Temperature T1

In [ ]:
df1 = pd.read_excel('C:/Users/onifa/Downloads/rest/T1.xlsx')
temp4 = df1.iloc[:, 11:17]
metacyte = df1.iloc[:, 17:130]
concat = [metacyte, temp4 ]
metacyte_temp = pd.concat(concat, axis=1)

In [ ]:
columns = metacyte_temp.columns
spearman_corr_matrix = pd.DataFrame(index=columns, columns=columns)
p_values_matrix = pd.DataFrame(index=columns, columns=columns)

for i in range(len(columns)):
    for j in range(i, len(columns)):
        col1, col2 = columns[i], columns[j]
        spearman_corr, p_value = spearmanr(metacyte_temp[col1], metacyte_temp[col2])
        spearman_corr_matrix.loc[col1, col2] = spearman_corr
        spearman_corr_matrix.loc[col2, col1] = spearman_corr
        p_values_matrix.loc[col1, col2] = p_value
        p_values_matrix.loc[col2, col1] = p_value

spearman_corr_matrix = spearman_corr_matrix.apply(pd.to_numeric, errors='coerce')

In [ ]:
significance_level = 0.05
significant_correlations = np.abs(p_values_matrix) < significance_level 

# Select only the upper triangle of the correlation matrix to avoid duplicates
upper_triangle = np.triu(np.ones(p_values_matrix.shape), k=1)

# Apply the significance filter and upper triangle mask
significant_upper_triangle = np.logical_and(significant_correlations, upper_triangle)

# Extract the indices of significant correlations
significant_indices = np.where(significant_upper_triangle)

# Create a DataFrame to store the results
significant_results = pd.DataFrame({
    'Variable1': spearman_corr_matrix.columns[significant_indices[1]],
    'Variable2': spearman_corr_matrix.columns[significant_indices[0]],
    'Correlation': spearman_corr_matrix.values[significant_indices],
    'P-Value': p_values_matrix.values[significant_indices]
})

In [ ]:
# Filter values above 0.7
filtered_df = significant_results[(significant_results['Correlation'] >= 0.5) | (significant_results['Correlation'] <= -0.5)]

# Sort by the 'correlation' column in descending order
sorted_df = filtered_df.sort_values(by='Correlation', ascending=False)

# Display the sorted DataFrame
print(sorted_df)

In [ ]:
mask = sorted_df['Variable1'].isin(temp4) & sorted_df['Variable2'].isin(metacyte)

# Use the mask to select rows containing both 'variable 1' and 'variable 2' in the metabolites list
filtered_df = sorted_df[mask]

# Display the filtered DataFrame
print(filtered_df)

In [ ]:
corr = {
    'Source': [
        'Mean Tskin', 'Mean Tskin', 'Muscle temp2' 
    ],
    'Target': [
        "Lysine",
 "MMP-2 (26)_T1",
 '1-Monopalmitin'
    ],
    'Correlation': [
        0.576418,
0.544383,
 0.522606
    ]
}


links_df = pd.DataFrame(corr)

In [ ]:
import holoviews as hv
from holoviews import opts, dim
import holoviews.plotting.bokeh
hv.extension('bokeh')
hv.output(size=200)

links_df = pd.DataFrame(corr)

# Sample nodes data
nodes = [
        'Mean Tskin', 'Mean Tskin', 'Muscle temp2',
        "Lysine", "MMP-2 (26)_T1", '1-Monopalmitin']

nodes_df = pd.DataFrame(nodes)
nodes_df.rename(columns={nodes_df.columns[0]: 'node'}, inplace=True)
node = hv.Dataset(pd.DataFrame(nodes_df['node']))

# Create HoloViews Chord Diagram
chord = hv.Chord((links_df, node)).select(value=(None, None))

#custom_colormap = {True: 'blue', False: 'red'}
#links_df['Positive'] = links_df['Value'] > 0
#edge_colors = links_df['Positive'].map(custom_colormap)


# Customize chart options
chord.opts(opts.Chord(cmap='Category20', 
                edge_cmap='Category20',
                edge_color=dim('Source').str(),
                labels='node',
                edge_line_width=4,
                node_color=dim('node').str(),
                label_text_font_size='12pt'))
